<a href="https://colab.research.google.com/github/pratik-poudel/datacrunch/blob/main/18th%20April%200.00%20lgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import gc
# pd.options.display.float_format| = "{:.2f}".format
pd.options.display.max_columns = 500

import sklearn
from sklearn import *
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.utils.testing import all_estimators
from sklearn.base import RegressorMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RepeatedKFold, cross_val_score, KFold
from sklearn.multioutput import RegressorChain


import requests
import time
from scipy import stats

import lightgbm as lgb
from xgboost import XGBRegressor

from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR

from collections import Counter


In [50]:
train_datalink_X = 'https://tournament.datacrunch.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.datacrunch.com/data/y_train.csv'
hackathon_data_link = 'https://tournament.datacrunch.com/data/X_test.csv'

train = pd.read_csv(train_datalink_X)
target = pd.read_csv(train_datalink_y)
test = pd.read_csv(hackathon_data_link)

In [51]:
print(train.shape, test.shape)

(6538, 15) (3436, 15)


In [52]:
train.head()

,Moons,id,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13
0,0,0x9818876384b58d3c,0.50,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
1,0,0xf9aeb187b2dad4f,0.50,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0
2,0,0x131db64190c7c8af,0.75,1.0,0.75,0.0,0.0,0.0,0.0,0.5,0.5,1.0,0.75,0.75,1.0
3,0,0xdbcbe7dd2f47b4ea,0.50,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.50,0.75,0.5
4,0,0x580e576eca4dc196,0.75,1.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0


In [53]:
target.head()

,target_r,target_g,target_b
0,0.25,0.25,0.25
1,0.00,0.00,0.00
2,0.50,0.50,0.50
3,0.00,0.00,0.00
4,0.75,1.00,0.75


In [54]:
feat= [f for f in train.columns if "Feature" in f]
print(feat)

['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6', 'Feature_7', 'Feature_8', 'Feature_9', 'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13']


In [55]:
train[feat].corrwith(target.target_r).abs().sort_values()

Feature_2     0.003135
Feature_3     0.008503
Feature_6     0.009482
Feature_7     0.009482
Feature_4     0.009940
Feature_5     0.009940
Feature_11    0.013092
Feature_12    0.014269
Feature_8     0.027604
Feature_9     0.027604
Feature_13    0.037069
Feature_1     0.043522
Feature_10    0.051558
dtype: float64

In [72]:
params = {
    "objective": "regression",
    "boosting": "gbdt",
    # "num_leaves": 1280,
    "learning_rate": 0.05,
    # "feature_fraction": 0.85,
    # "reg_lambda": 2,
    "metric": "rmse",
}
X= train[feat].copy()
kf = KFold(n_splits=10)
models = []
for train_index,test_index in kf.split(X):
    print(train_index.shape)
    train_features = X.loc[train_index]
    train_target = target.target_r.loc[train_index]
    
    test_features = X.loc[test_index]
    test_target = target.target_r.loc[test_index]
    
    d_training = lgb.Dataset(train_features, label=train_target,free_raw_data=False)
    d_test = lgb.Dataset(test_features, label=test_target, free_raw_data=False)
    
    model = lgb.train(params, train_set=d_training, num_boost_round=1000, valid_sets=[d_training,d_test], verbose_eval=25, early_stopping_rounds=50)
    models.append(model)
    del train_features, train_target, test_features, test_target, d_training, d_test
    gc.collect()

(5884,)
Training until validation scores don't improve for 50 rounds.
[25]	training's rmse: 0.341945	valid_1's rmse: 0.370283
[50]	training's rmse: 0.335812	valid_1's rmse: 0.37394
Early stopping, best iteration is:
[2]	training's rmse: 0.35093	valid_1's rmse: 0.367613
(5884,)
Training until validation scores don't improve for 50 rounds.
[25]	training's rmse: 0.342174	valid_1's rmse: 0.368016
[50]	training's rmse: 0.335722	valid_1's rmse: 0.370499
Early stopping, best iteration is:
[3]	training's rmse: 0.350548	valid_1's rmse: 0.36643
(5884,)
Training until validation scores don't improve for 50 rounds.
[25]	training's rmse: 0.341846	valid_1's rmse: 0.372861
[50]	training's rmse: 0.335807	valid_1's rmse: 0.375291
Early stopping, best iteration is:
[1]	training's rmse: 0.351034	valid_1's rmse: 0.371137
(5884,)
Training until validation scores don't improve for 50 rounds.
[25]	training's rmse: 0.343104	valid_1's rmse: 0.356024
[50]	training's rmse: 0.336131	valid_1's rmse: 0.357599
Early

In [73]:
models

In [79]:
train[train.Moons >= 12]

,Moons,id,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13
5582,12,0xdc90ea2ea93fec6a,0.25,0.25,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.50,0.75,0.25,0.50
5583,12,0xcd5e19423e5ebaa1,0.00,1.00,0.00,0.0,0.0,0.75,0.75,1.00,1.00,0.50,0.75,1.00,0.50
5584,12,0x2a8794e147c18939,0.25,0.75,0.50,0.0,0.0,0.00,0.00,0.25,0.25,0.00,0.00,0.00,0.00
5585,12,0xefee997e4355494a,0.75,0.75,0.50,0.0,0.0,0.00,0.00,0.25,0.25,0.25,0.50,0.25,0.75
5586,12,0x48656926a0729607,0.75,0.75,0.25,0.0,0.0,0.00,0.00,0.00,0.00,1.00,0.25,0.75,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6533,13,0xee1509f813ae5a29,0.25,0.50,0.50,1.0,1.0,0.75,0.75,0.75,0.75,0.00,0.00,0.00,0.00
6534,13,0x5be7f5f0950653aa,0.00,0.25,0.25,1.0,1.0,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00
6535,13,0x69cf43587a77375b,0.50,0.50,0.50,1.0,1.0,1.00,1.00,1.00,1.00,0.25,0.50,0.75,0.50
6536,13,0xd3618b1bcfac1137,0.25,0.75,1.00,1.0,1.0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00


In [77]:
train

,Moons,id,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13
0,0,0x9818876384b58d3c,0.50,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0,0xf9aeb187b2dad4f,0.50,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0,0x131db64190c7c8af,0.75,1.00,0.75,0.0,0.0,0.00,0.00,0.50,0.50,1.00,0.75,0.75,1.00
3,0,0xdbcbe7dd2f47b4ea,0.50,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.50,0.50,0.75,0.50
4,0,0x580e576eca4dc196,0.75,1.00,0.75,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6533,13,0xee1509f813ae5a29,0.25,0.50,0.50,1.0,1.0,0.75,0.75,0.75,0.75,0.00,0.00,0.00,0.00
6534,13,0x5be7f5f0950653aa,0.00,0.25,0.25,1.0,1.0,0.25,0.25,0.00,0.00,0.00,0.00,0.00,0.00
6535,13,0x69cf43587a77375b,0.50,0.50,0.50,1.0,1.0,1.00,1.00,1.00,1.00,0.25,0.50,0.75,0.50
6536,13,0xd3618b1bcfac1137,0.25,0.75,1.00,1.0,1.0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,0.00


In [82]:
!pip install optuna

  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=47f8e6c5d85b99554107c29938a00b644a0dc08b73a6226aa73d7ee40b5ef511
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [95]:
import optuna

def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred))[0]
    # print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    return score

train_all= pd.concat([train, target], axis=1)
test_data = train_all[train_all['Moons'] > 11]

targets= ['target_r', 'target_g', 'target_b']


train = train_all[train_all['Moons'] <=11 ]

X_train, X_test, y_train, y_test = train_test_split(train[feat], train[targets], test_size=0.25, shuffle=False)

tr_data_r = lgb.Dataset(X_train, y_train.target_r)
va_data_r = lgb.Dataset(X_test, y_test.target_r)

tr_data_g = lgb.Dataset(X_train, y_train.target_g)
va_data_g = lgb.Dataset(X_test, y_test.target_g)

tr_data_b = lgb.Dataset(X_train, y_train.target_b)
va_data_b = lgb.Dataset(X_test, y_test.target_b)


N_TRIALS = 500
def objective(trial):    
    
    params = {
    'num_leaves': trial.suggest_int('num_leaves', 4, 64),
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'max_depth': trial.suggest_int('max_depth', 4, 16),
    'min_child_weight': trial.suggest_int('min_child_weight', 1, 16),
    'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
    'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
    'bagging_freq': trial.suggest_int('bagging_freq', 1, 8),
    'min_child_samples': trial.suggest_int('min_child_samples', 4, 80),
    'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 1.0),
    'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 1.0),
    'learning_rate' : trial.suggest_uniform('learning_rate', 0.0001, 0.99),
    'max_bin': trial.suggest_int('max_bin', 50, 900),
    'early_stopping_rounds': 100, 
    }


    model_r = lgb.train(params, tr_data_r, valid_sets=[tr_data_r, va_data_r], verbose_eval=None, num_boost_round=10000)
    model_g = lgb.train(params, tr_data_g, valid_sets=[tr_data_g, va_data_g], verbose_eval=None, num_boost_round=10000)
    model_b = lgb.train(params, tr_data_b, valid_sets=[tr_data_b, va_data_b], verbose_eval=None, num_boost_round=10000)

    
    score_r = scorer(test_data.target_r, model_r.predict(test_data[feat]))
    score_g = scorer(test_data.target_g, model_g.predict(test_data[feat]))
    score_b = scorer(test_data.target_b, model_b.predict(test_data[feat]))

    # print((score_r+score_g+score_b)/ 3)
    score = (score_r+score_g+score_b)/ 3

    print(score)
    
    from google.colab import output
    output.clear()
    return score
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar= True)
display(study.best_params)
display(study.best_trial.value)

[I 2021-04-18 16:00:01,650] Trial 499 finished with value: 0.09549372786373174 and parameters: {'num_leaves': 11, 'max_depth': 14, 'min_child_weight': 5, 'feature_fraction': 0.9597589238790629, 'bagging_fraction': 0.7058262831331051, 'bagging_freq': 7, 'min_child_samples': 75, 'lambda_l1': 0.001119049799203641, 'lambda_l2': 2.356290557744705e-07, 'learning_rate': 0.4979424945406407, 'max_bin': 415}. Best is trial 398 with value: 0.12562811462758292.



{'bagging_fraction': 0.7123038688804579,
 'bagging_freq': 7,
 'feature_fraction': 0.8114760919725509,
 'lambda_l1': 0.00716353507398451,
 'lambda_l2': 5.091157987021881e-07,
 'learning_rate': 0.7930891050529489,
 'max_bin': 506,
 'max_depth': 11,
 'min_child_samples': 71,
 'min_child_weight': 7,
 'num_leaves': 9}

0.12562811462758292

In [96]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    
    'bagging_fraction': 0.637925746336356,
 'bagging_freq': 7,
 'feature_fraction': 0.8232500043587935,
 'lambda_l1': 2.1371835507271136e-08,
 'lambda_l2': 0.0009127283964828737,
 'learning_rate': 0.9885679489935546,
 'max_bin': 465,
 'max_depth': 14,
 'min_child_samples': 48,
 'min_child_weight': 8,
 'num_leaves': 4}


train_datalink_X = 'https://tournament.datacrunch.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.datacrunch.com/data/y_train.csv'
hackathon_data_link = 'https://tournament.datacrunch.com/data/X_test.csv'

train = pd.read_csv(train_datalink_X)
target = pd.read_csv(train_datalink_y)
test = pd.read_csv(hackathon_data_link)

feat= [f for f in train.columns if "Feature" in f]

def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred))[0]
    # print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))
    return score


X_train, X_test, y_train, y_test = train_test_split(train[feat], target[targets], test_size=0.25, shuffle=False)
tr_data_r = lgb.Dataset(X_train, y_train.target_r)
va_data_r = lgb.Dataset(X_test, y_test.target_r)

tr_data_g = lgb.Dataset(X_train, y_train.target_g)
va_data_g = lgb.Dataset(X_test, y_test.target_g)

tr_data_b = lgb.Dataset(X_train, y_train.target_b)
va_data_b = lgb.Dataset(X_test, y_test.target_b)

model_r = lgb.train(params, tr_data_r, valid_sets=[tr_data_r, va_data_r], verbose_eval=2000, num_boost_round=10000, early_stopping_rounds=100)
model_g = lgb.train(params, tr_data_g, valid_sets=[tr_data_g, va_data_g], verbose_eval=2000, num_boost_round=10000, early_stopping_rounds=100)
model_b = lgb.train(params, tr_data_b, valid_sets=[tr_data_b, va_data_b], verbose_eval=2000, num_boost_round=10000, early_stopping_rounds=100)


prediction= pd.DataFrame()
prediction['target_r'] = model_r.predict(test[feat])
prediction['target_g'] = model_g.predict(test[feat])
prediction['target_b'] = model_b.predict(test[feat])

Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	training's rmse: 0.344634	valid_1's rmse: 0.375596
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[1]	training's rmse: 0.351965	valid_1's rmse: 0.345097
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[2]	training's rmse: 0.351333	valid_1's rmse: 0.339932


In [94]:
prediction.target_r.value_counts()

0.516381    1325
0.455984     456
0.475879     285
0.437962     268
0.590431     257
0.509074     226
0.528856     160
0.521549     152
0.450436     152
0.530033      74
0.536276      69
0.468458      12
Name: target_r, dtype: int64

In [ ]:
prediction.target_b.value_counts()

In [100]:
API_KEY = "0GIskIC9Rd1I8sMw6nStutbIbMx7AaZzAyNfuMOVAVWgJtNa1IVQHZAD3WPj" # <- HERE

r = requests.post("https://tournament.datacrunch.com/api/submission",
    files = {
        "file": ("x", prediction.to_csv().encode('ascii'))
    },
    data = {
        "apiKey": API_KEY
    },
)

if r.status_code == 200:
    print("Submission submitted :)")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("You can only submit during rounds eg: Friday 7pm GMT+1 to Sunday midnight GMT+1.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 404:
    print("ERR: Unknown API Key")
    print("You should check that the provided API key is valid and is the same as the one you've received by email.")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
elif r.status_code == 401:
    print("ERR: Your email hasn't been verified")
    print("Please verify your email or contact a cruncher.")
elif r.status_code == 429:
    print("ERR: Too many submissions")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

Submission submitted :)
